### MANDATORY ASSIGNMENT 2

In [1]:
from sklearn import datasets

In [2]:
import numpy as np

In [3]:
iris = datasets.load_iris()

In [4]:
X = iris.data
Y = iris.target

#### Task 1) data exploration

In [5]:
len(X)

150

In [6]:
print(X.shape, Y.shape)

(150, 4) (150,)


In [7]:
print(np.min(X), np.max(X))
print(np.min(Y), np.max(Y))

0.1 7.9
0 2


##### 2) Encoding and pre-processing

In [8]:
#at first we normalize the data from 0 to pi, and then implement angle encoding
from sklearn.preprocessing import MinMaxScaler
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer, AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter
from qiskit_algorithms.optimizers import GradientDescent
import random

In [9]:
scaler = MinMaxScaler(feature_range=(0, np.pi))
X = scaler.fit_transform(X)

In [10]:
def angle_encoding(qc, sample):
    for qubit in range(len(qc.qubits)):
        qc.rx(sample[qubit], qubit)

    

##### 3) choosing Loss function

In [11]:
from sklearn.metrics import root_mean_squared_error


##### 4) splitting data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42) # 70% training 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # 15% validation, 15% testing

In [54]:
num_qubits = 4
num_layers = 3
parameters = [Parameter(f"theta_{i}") for i in range(num_layers * num_qubits)]

In [16]:
int('1111', 2)

15

In [49]:
theta = '\u03B8'

def real_amplitudes(data_point,parameters, layers = 2):
    qc = QuantumCircuit(num_qubits)
    angle_encoding(qc, data_point)

    param_index = 0

    for layer in range(layers):
        for qubit in range(len(qc.qubits)):
            qc.ry(parameters[param_index], qubit)
            param_index += 1
        qc.barrier()
        
        for qubit in range(len(qc.qubits)-1):
            qc.cx(qubit, qubit+1)
        qc.barrier()

    return qc
    

In [59]:
def data_decoding(output):
    return int(output, 2) % 3 #

In [60]:
def cost_function(updated_params):
    qc = real_amplitudes(X_train[0], parameters, layers = num_layers)
    qc.assign_parameters(updated_params)

    backend = AerSimulator()
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=1000)
    result = job.result()
    counts = result.get_counts(qc)

    prediction = data_decoding(max(counts, key=counts.get))

    cost = root_mean_squared_error(np.array([[y_train[0]]]), np.array([[prediction]]))
    print(f"Parameters: {updated_params} cost: {cost}")
    return cost
    

In [56]:
print(X_train[0], y_train[0])

[1.04719755 0.52359878 1.43767799 1.17809725] 1


In [58]:
initial_parameters = np.zeros(num_layers * num_qubits)

# Gradient Descent optimizer
optimizer = GradientDescent(maxiter=100, learning_rate=0.1)

# Optimize the parameters
result = optimizer.minimize(fun=cost_function, x0=initial_parameters)

print("Optimized Parameters:", result.x)
print("Minimum Cost:", result.fun)


AerError: 'circuits have parameters but parameter_binds is not specified.'

In [34]:
qc = QuantumCircuit(num_qubits)
qc = real_amplitudes(qc, X_train[0], result.x, layers = num_layers)

backend = AerSimulator()
tqc = transpile(qc, backend)
job = backend.run(tqc, shots=1000)
result = job.result()
counts = result.get_counts(qc)

prediction = data_decoding(max(counts, key=counts.get))
print(prediction)

0
